In [22]:
# -*-  coding: utf-8 -*-
import os, sys
import jieba, codecs, math
import jieba.posseg as pseg
import paddle
 
names = {}            # 姓名字典
relationships = {}    # 关系字典
lineNames = []        # 每段内人物关系

In [23]:
jieba.load_userdict('../resource/people.txt')
paddle.enable_static()
jieba.enable_paddle()

Paddle enabled successfully......


In [24]:
with codecs.open("finaldata.txt", "r", "utf8") as f:
    for line in f.readlines():
        poss = pseg.cut(line)        # 分词并返回该词词性
        lineNames.append([])        # 为新读入的一段添加人物名称列表
        for w in poss:
            if w.flag != "nr"  :#or len(w.word) < 2:
                continue            # 当分词长度小于2或该词词性不为nr时认为该词不为人名
            lineNames[-1].append(w.word)        # 为当前段的环境增加一个人物
            if names.get(w.word) is None:
                names[w.word] = 0
                relationships[w.word] = {}
            names[w.word] += 1                    # 该人物出现次数加 1

In [25]:
for name, times in names.items():
    print(name, times)

奥 1
杨恩 1
费格伦 2
女儿 10
梅莉丝 3
维 1
道奇 1
谢谢 12
凯莉 2
芬恩 2
妈妈 4
张开嘴 1
谢谢您 2
史托维克 2
特蕾莎 1
马上会 1
胡闹 1
安全带 1
平顺 1
赛奈 1
陆军 1
阿姨 1
艾米许 3
祖母 1
通耶 1
古董 3
耶 1
梅兰妮 3
欧多利欧 13
祖先 1
文明 1
宝贝 4
美的 1
时髦 1
滑进来 1
付元 1
阿贝 1
儿子 2
罗伯特 2
车库里 1
塞勒斯 1
小塞 3
言语 1
老二 1
太久 1
安静 3
真是太 1
寇黑塔 2
那本书 1
罗 1
祝福 1


In [26]:
for line in lineNames:                    # 对于每一段
    for name1 in line:                    
        for name2 in line:                # 每段中的任意两个人
            if name1 == name2:
                continue
            if relationships[name1].get(name2) is None:        # 若两人尚未同时出现则新建项
                relationships[name1][name2]= 1
            else:
                relationships[name1][name2] = relationships[name1][name2]+ 1        # 两人共同出现次数加 1



In [27]:
with codecs.open("node.txt", "w", "utf8") as f:
    f.write("Id Label Weight\r\n")
    for name, times in names.items():
        f.write(name + " " + name + " " + str(times) + "\r\n")
 
with codecs.open("edge.txt", "w", "utf8") as f:
    f.write("Source Target Weight\r\n")
    for name, edges in relationships.items():
        for v, w in edges.items():
            if w > 3:
                f.write(name + " " + v + " " + str(w) + "\r\n")